# **SlopeEXCL - Slope-based Exclusion**
This notebook contains Python functions to calculate slope-based exclusions for wind and solar energy site suitability.

The key functionalities include:
- Preprocessing a Digital Elevation Model (DEM) to compute slope and aspect.
- Creating exclusion masks for unsuitable areas based on slope thresholds.
- Saving the exclusion results as raster files.

In [ ]:
import os
import numpy as np
import rasterio
from scipy.ndimage import sobel, gaussian_filter

## **Function Definitions**
These functions perform slope and aspect calculations, create exclusion masks, and save the results.

In [2]:
def process_dem(dem_path, sigma=1):
    """
    Process a DEM to calculate slope and aspect.
    
    Args:
        dem_path (str): Path to the DEM file.
        sigma (float): Standard deviation for Gaussian smoothing.
    
    Returns:
        tuple: Slope, aspect arrays, raster profile, and NoData value.
    """
    with rasterio.open(dem_path) as src:
        dem = src.read(1)
        profile = src.profile
        transform = src.transform
        nodata = src.nodata

        latitude = transform[5]
        pixel_size_m = 90
        latitude_correction = np.cos(np.radians(latitude))
        pixel_size_x = pixel_size_m * latitude_correction
        pixel_size_y = pixel_size_m * latitude_correction

        smoothed_dem = gaussian_filter(dem, sigma=sigma)
        gradient_x = sobel(smoothed_dem, axis=1) / (2 * pixel_size_x)
        gradient_y = sobel(smoothed_dem, axis=0) / (2 * pixel_size_y)

        slope = np.degrees(np.arctan(np.sqrt(gradient_x**2 + gradient_y**2)))
        aspect = np.degrees(np.arctan2(-gradient_y, gradient_x))
        aspect = np.where(aspect < 0, 360 + aspect, aspect)

    return slope, aspect, profile, nodata

### **Calculate Exclusion Percentages**

In [3]:
def calculate_percentage_excluded(mask):
    """
    Calculate the percentage of excluded points.
    
    Args:
        mask (numpy.ndarray): Boolean array indicating excluded areas.
    
    Returns:
        float: Percentage of excluded points.
    """
    total_points = np.prod(mask.shape)
    excluded_points = np.sum(mask)
    return (excluded_points / total_points) * 100

### **Create Combined Exclusion Mask**

In [4]:
def create_combined_exclusion(dem_path, output_name, solar_nea, solar_s, wind_thresh, sigma=1):
    """
    Generate combined exclusion mask for solar and wind.
    
    Args:
        dem_path (str): Path to the DEM file.
        output_name (str): Name of the output file.
        solar_nea (float): Slope threshold for north-east-west solar aspects.
        solar_s (float): Slope threshold for south solar aspects.
        wind_thresh (float): Slope threshold for wind exclusions.
        sigma (float): Standard deviation for Gaussian smoothing.
    """
    slope, aspect, profile, nodata = process_dem(dem_path, sigma)

    north_east_west_mask = (aspect >= 45) & (aspect <= 135) | (aspect >= 225) & (aspect <= 315)
    south_mask = (aspect > 135) & (aspect < 225)
    solar_exclusion = (
        ((north_east_west_mask) & (slope >= solar_nea)) |
        ((south_mask) & (slope >= solar_s))
    )
    wind_exclusion = slope >= wind_thresh
    combined_exclusion = solar_exclusion | wind_exclusion

    solar_percentage = calculate_percentage_excluded(solar_exclusion)
    wind_percentage = calculate_percentage_excluded(wind_exclusion)
    combined_percentage = calculate_percentage_excluded(combined_exclusion)

    print(f"Percentage of points excluded for solar: {solar_percentage:.2f}%")
    print(f"Percentage of points excluded for wind: {wind_percentage:.2f}%")
    print(f"Percentage of points excluded (combined): {combined_percentage:.2f}%")

    save_exclusion_raster(combined_exclusion, profile, output_name)

### **Create Solar Exclusion**

In [5]:
def create_solar_exclusion(dem_path, output_name, solar_nea, solar_s, sigma=1):
    """
    Create and save a solar exclusion mask.

    This function generates an exclusion mask for solar PV development based on terrain slope and aspect 
    and saves the result to a raster file.

    Args:
        dem_path (str): Path to the input DEM file (e.g., a GeoTIFF).
        output_name (str): Name of the output raster file.
        solar_nea (float): Slope threshold for north, east, and west-facing slopes for solar PV.
        solar_s (float): Slope threshold for south-facing slopes for solar PV.
        sigma (float): Standard deviation for the Gaussian filter applied to smooth the DEM.

    Returns:
        None
    """
    slope, aspect, profile, nodata = process_dem(dem_path, sigma)

    north_east_west_mask = (aspect >= 45) & (aspect <= 135) | (aspect >= 225) & (aspect <= 315)
    south_mask = (aspect > 135) & (aspect < 225)
    solar_exclusion = (
        ((north_east_west_mask) & (slope >= solar_nea)) |
        ((south_mask) & (slope >= solar_s))
    )

    solar_percentage = calculate_percentage_excluded(solar_exclusion)
    print(f"Percentage of points excluded for solar: {solar_percentage:.2f}%")

    save_exclusion_raster(solar_exclusion, profile, output_name)

### **Create Wind Exclusion**

In [6]:

def create_wind_exclusion(dem_path, output_name, wind_thresh, sigma=1):
    """
    Create and save a wind exclusion mask.

    This function generates an exclusion mask for wind development based on terrain slope 
    and saves the result to a raster file.

    Args:
        dem_path (str): Path to the input DEM file (e.g., a GeoTIFF).
        output_name (str): Name of the output raster file.
        wind_thresh (float): Slope threshold for wind exclusion.
        sigma (float): Standard deviation for the Gaussian filter applied to smooth the DEM.

    Returns:
        None
    """
    slope, _, profile, nodata = process_dem(dem_path, sigma)

    wind_exclusion = slope >= wind_thresh

    wind_percentage = calculate_percentage_excluded(wind_exclusion)
    print(f"Percentage of points excluded for wind: {wind_percentage:.2f}%")

    save_exclusion_raster(wind_exclusion, profile, output_name)

### **Save Exclusion Raster**

In [7]:

def save_exclusion_raster(mask, profile, output_name):
    """
    Save an exclusion mask to a raster file.

    This function writes a boolean mask to a GeoTIFF file, ensuring the mask is saved 
    as a single-layer raster with the appropriate metadata.

    Args:
        mask (numpy.ndarray): Boolean array representing the exclusion mask.
        profile (dict): Raster profile for the output file (e.g., spatial resolution, projection).
        output_name (str): Name of the output raster file.

    Returns:
        None
    """
    output_folder = "output"
    os.makedirs(output_folder, exist_ok=True)

    output_path = os.path.join(output_folder, output_name)

    profile.update(
        dtype=rasterio.uint8,
        count=1,
        compress='deflate',
        nodata=0
    )
    
    with rasterio.open(output_path, 'w', **profile) as dst:
        dst.write(mask.astype(rasterio.uint8), 1)

    print(f"Exclusion raster saved to {output_path}")

## **Example Execution**
Below is an example of how to use the defined functions to generate a combined exclusion mask.


In [9]:
# Define paths and parameters
dem_path = "data/dem.tif"  # Path to the DEM file
output_name = 'exclusion_combined.tif'  # Output file

# Default thresholds for solar and wind exclusions
solar_nea = 6.28  # Threshold for north-east-west solar slopes (degrees)
solar_s = 33  # Threshold for south solar slopes (degrees)
wind_thresh = 8.53  # Threshold for wind slopes (degrees)
sigma = 1  # Gaussian smoothing factor

# Generate the combined exclusion mask
create_combined_exclusion(dem_path, output_name, solar_nea, solar_s, wind_thresh, sigma)

Percentage of points excluded for solar: 16.76%
Percentage of points excluded for wind: 23.79%
Percentage of points excluded (combined): 24.42%
Exclusion raster saved to output\exclusion_combined.tif


After running the code, the exclusion raster will be saved to the `output` directory.